## Workflow
```
1. Create a new email address
2. Add email address to persona-email-map.json file 
3. Create Amazon account with that email
    a. At times it is difficult to create an Amazon account without a phone number
4. Create a fresh browser profile and update preferences as mentioned below
5. Reset Amazon Echo
6. Sign in to Protonmail, Amazon, and alexa.amazon.com
7. Install skills!
```

### Manually updated preferences in profile (Important!):
```
1. Select custom privacy protection and disable all
2. Disable auto updates
3. Disable default browser check
4. Disable recommended by pocket 
```
------
```
1. extensions.experiments.enabled --> True
2. xpinstall.signatures.required --> False
3. network.cookie.cookieBehavior --> 0 (auto)
4. browser.shell.checkDefaultBrowser --> False (auto)
```

### Persona popularity
1. https://www.statista.com/statistics/301868/distribution-digital-ad-spend-by-industry-channel-usa/
2. https://www.statista.com/statistics/275506/top-advertising-categories-in-the-us/
3. https://www.marketingcharts.com/charts/top-10-advertising-categories-in-q1-2020

### Install dependencies

In [ ]:
# ! pip3 install selenium
# brew install geckodriver
# Download Firefox Nightly
# https://firefox-ci-tc.services.mozilla.com/api/index/v1/task/gecko.v2.mozilla-release.revision.cda19fb46018c9cb8dc5e6127ebb2cfd04aa9be4.firefox.macosx64-add-on-devel/artifacts/public/build/target.dmg

### [TODO] 

1. Programatically create new profiles. Borrow code from OpenWPM: https://github.com/mozilla/OpenWPM/blob/9b4d3582bc3bdb4e319874a427d63ff14feadd2c/openwpm/deploy_browsers/deploy_firefox.py

In [17]:
from selenium import webdriver
import time

from selenium.webdriver import FirefoxOptions
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import utilities
import os

### Helper functions

In [18]:
FIREFOX_EXE_PATH = '/Applications/Nightly.app/Contents/MacOS/firefox-bin'
GECKO_PATH = '/opt/homebrew/bin/geckodriver'

# skill with no reviews
SIGNIN_PAGE = 'https://www.amazon.com/Sarim-Studios-CurrentBitcoin/dp/B01N9SS2LI/ref=sr_1_3641'


DATA_DIR = '/Users/umariqbal/Documents/voice-assistant-central/' 
EXTENSION_PATH = os.path.join(DATA_DIR, 'browser-extension.zip')

CREDENTIALS_ADDR = os.path.join(DATA_DIR,'data/persona-email-map.json')
SKILLS_ADDR = os.path.join(DATA_DIR,'data/grouped_skills.json')

#### Update persona name

In [19]:
PERSONA = 'SmartHome'

In [20]:
def get_webdriver(profile_path, gecko_path, exe_path, ext_loc):
    options = FirefoxOptions()
    options.add_argument('-profile')
    options.add_argument(profile_path)
    
    firefox_capabilities = DesiredCapabilities.FIREFOX
    firefox_capabilities['marionette'] = True

    
    firefox_profile = webdriver.FirefoxProfile()
    firefox_bin = FirefoxBinary(firefox_path=exe_path)
    driver = webdriver.Firefox(firefox_binary= firefox_bin,
                               capabilities=firefox_capabilities, 
                               options=options, 
                               executable_path=gecko_path, 
                               service_args=["--marionette-port", "2828"])

    driver.install_addon(ext_loc, temporary=True)
    
    return driver


def quit_driver(driver):
    try:
        driver.quit()
    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        
        
def signin(driver, singin_page, email, password):
    try:
        driver.get(singin_page)
        time.sleep(3)

        signin_button = driver.find_element_by_id('a-autoid-0-announce')
        signin_button.click()
        time.sleep(3)
        
        try:
            email_input = driver.find_element_by_id('ap_email')

        except Exception:
            ## Assume that the user is already signed in 
            print('Already signed in. Returning.')
            return True
            
        email_input.send_keys(email)
        time.sleep(3)
        
        continue_button = driver.find_element_by_id('continue')
        continue_button.click()
        time.sleep(3)
        
        password_input = driver.find_element_by_id('ap_password')
        password_input.send_keys(password)
        time.sleep(3)
        
        signin_button = driver.find_element_by_id('signInSubmit')
        signin_button.click()
        time.sleep(3)
        
    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        return False
    
    return True


def install_skill(driver, url, perm = False):
    try:
        driver.get(url)
        time.sleep(3)
        
        skill_enable_button = driver.find_element_by_id("a2s-skill-enable-button")
        skill_enable_button.click()
        time.sleep(3)
        
        if perm:
            try: 
                permission_button = driver.find_element_by_xpath("//input[@type='submit'][@class='a-button-input'][not(@aria-labelledby)]")
                permission_button.click()
                time.sleep(3)
       
            except Exception:
                print('Something went wrong when enabling permissions')
                return False, 'perm'
            

    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        return False, 'sim'
    
    return True, 'sim'

In [21]:
def train_persona(profile_path, email, password, skills_json, 
                  no_skills_to_install=50, signin_page=SIGNIN_PAGE,  
                  gecko_path=GECKO_PATH, exe_path=FIREFOX_EXE_PATH, ext_loc=EXTENSION_PATH):
    
    driver = get_webdriver(profile_path, gecko_path, exe_path, ext_loc)
    print('WebDriver registered')
    
    signin_status = signin(driver, signin_page, email, password)

    total_installed = 0
    if signin_status:
        print('Signed in.')
        for skill in skills_json:
            
            skill_perm = all_skills[skill]['Skill_permission']
            skill_url = all_skills[skill]['Skill_link']
            
            if len(skill_perm) > 0:
                skill_perm = True
            else:
                skill_perm = False
                
            
            install_status = install_skill(driver, skill_url, skill_perm)
            
            # Log succesfully and partially installed skills
            if install_status[0]:
                total_installed += 1
            
            if total_installed >= no_skills_to_install:
                break
    else:
        print('Could not sign in. Stopping the process')
        quit_driver(driver)
        
    
    quit_driver(driver)
    print(total_installed, len(skills_json))
    print('Installed %f skills.', (total_installed / len(no_skills_to_install)) * 100.0)

### Read email, password, and profile for the persona

In [22]:
credential = utilities.read_json(CREDENTIALS_ADDR)

email = credential[PERSONA]['email']
password = credential[PERSONA]['password']
profile_addr = credential[PERSONA]['profile']

In [23]:
email, password, profile_addr

('phillip-phillips-6@protonmail.com',
 'change.me',
 '/Users/umariqbal/Library/Application Support/Firefox/Profiles/dslzuepf.smart-home')

### Read top skills for the selected persona

In [24]:
all_skills = utilities.read_json(SKILLS_ADDR)
all_skills = all_skills[PERSONA]
# all_skills = all_skills['Local']

In [25]:
# profile_addr = '/Users/umariqbal/Library/Application Support/Firefox/Profiles/y7zzv5nf.va-test'
# email = "phillip-phillips-1@protonmail.com"
# password = "change.me"
# profile_addr = '/Users/umariqbal/Library/Application Support/Firefox/Profiles/nf5umchy.va-night-test'

train_persona(profile_addr, email, password, all_skills)

WebDriver registered
Already signed in. Returning.
Signed in.
50 2417
Installed %f skills. 2.068680182043856
